# Pacote

In [1]:
import pandas as pd
import chardet
import glob
from unidecode import unidecode
import os
import Levenshtein as lev
import csv
import sys

# PySpark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SQLContext, SparkConf
import findspark

# geopy
from geopy.distance import geodesic

# Configuração do pandas
pd.set_option('display.max_columns', None)

# Dicionário com bibliotecas e suas versões
libraries = {
    "pandas": pd.__version__,
    "chardet": chardet.__version__,
    "glob": "Built-in library (Python Standard Library)",
    "unidecode": unidecode.__name__,
    "os": "Built-in library (Python Standard Library)",
    "Levenshtein": lev.__version__,
    "csv": "Built-in library (Python Standard Library)",
    "sys": sys.version,
    "pyspark": pyspark.__version__,
    "findspark": findspark.__version__,
    "geopy": geodesic.__module__.split('.')[0] + " (" + "Version not directly available, likely from 'pip list')",
}

# Exibindo as versões
for lib, version in libraries.items():
    print(f"{lib}: {version}")


pandas: 2.1.4
chardet: 5.2.0
glob: Built-in library (Python Standard Library)
unidecode: unidecode_expect_ascii
os: Built-in library (Python Standard Library)
Levenshtein: 0.23.0
csv: Built-in library (Python Standard Library)
sys: 3.11.0 (main, Oct 24 2022, 18:26:48) [MSC v.1933 64 bit (AMD64)]
pyspark: 3.5.0
findspark: 2.0.1
geopy: geopy (Version not directly available, likely from 'pip list')


In [2]:
import sys

print(os.environ.get("SPARK_HOME"))
print(os.environ.get("HADOOP_HOME"))
print(os.environ.get("JAVA_HOME"))

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

C:\Users\pedro\spark-3.5.0-bin-hadoop3
C:\Users\pedro\hadoop3.0
C:\Program Files\Java\jdk1.8.0_202


In [3]:
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.types import *
from pyspark.sql.functions import lower, upper,row_number,isnan, when, count, col, coalesce, broadcast, regexp_replace, regexp_extract, lit, countDistinct
from pyspark.sql import functions as F, Window, Row
from pyspark.sql.functions import *
#from functools import reduce

#Pyspark
import py4j
from pyspark import SparkContext,SQLContext,SparkConf,StorageLevel

## Pacotes para configurar sessão no spark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
                            
## Pacote para localizar o path spark 
import findspark

from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
import unicodedata

## Spark session

In [4]:
# Usa todos os núcleos disponíveis na máquina local.
# Define o nome da aplicação.
# Número de núcleos alocados para o driver Spark.
# Quantidade de memória alocada para o driver Spark.
# Nível de paralelismo padrão para todas as transformações em RDDs.
# Número de partições para usar quando fazer operações de shuffle.
# Número de instâncias do executor para iniciar.
# Número de núcleos para usar por executor.
# Quantidade de memória alocada para cada executor.
# Fração da heap do executor para armazenamento e execução.
# Proporção da memória de execução acima da qual o armazenamento será despejado para o disco.
# Habilita o uso de memória fora do heap.
# Tamanho da memória fora do heap alocada para o Spark.
# Tamanho máximo dos resultados do driver.
# Memória adicional alocada por executor.
# Habilita a avaliação antecipada e a visualização dos DataFrames no Spark SQL REPL.
# Número máximo de linhas para mostrar quando a avaliação antecipada está habilitada.
# Tamanho máximo do buffer para serialização Kryo.
# Tamanho máximo das tabelas na realização do broadcast join 
# Usa KryoSerializer para serialização, oferecendo melhor desempenho.
# Classe de registrator Kryo para registrar classes personalizadas com Kryo.
# Comprime os dados shuffle para economizar espaço em disco.
# Define o nível de armazenamento para RDDs persistidos, usando tanto a memória quanto o disco.
# Comprime RDDs armazenados em memória.

spark = (SparkSession.builder 
    .master("local[*]") 
    .appName("Spark Optimization")   
    .config("spark.driver.cores", "2")   
    .config("spark.driver.memory", "8g")   
    .config("spark.default.parallelism", "24")   
    .config("spark.sql.shuffle.partitions", "24")   
    .config("spark.executor.instances", "3")   
    .config("spark.executor.cores", "2")   
    .config("spark.executor.memory", "10g")   
    .config("spark.memory.fraction", "0.6")  
    .config("spark.memory.storageFraction", "0.5")   
    .config("spark.memory.offHeap.enabled", "true")   
    .config("spark.memory.offHeap.size", "4g")   
    .config("spark.driver.maxResultSize", "4g")   
    .config("spark.executor.memoryOverhead", "2g")   
    .config("spark.sql.repl.eagerEval.enabled", True)   
    .config("spark.sql.repl.eagerEval.maxNumRows", 10)  
#    .config("spark.kryoserializer.buffer.max", "512m")  
    .config("spark.sql.autoBroadcastJoinThreshold", "400m")   
#    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")   
#    .config("spark.kryo.registrator", "MyKryoRegistrator")   
    .config("spark.shuffle.compress", "true")   
    .config("spark.storage.level", "MEMORY_AND_DISK")   
    .config("spark.rdd.compress", "true")   
    .getOrCreate())

In [5]:
spark

# Carregando dados

## Df meteorologia

In [6]:
df = (spark.read.parquet("DADOS_METEROLOGIA/DADOS_METEOROLOGICOS_TRATADOS/dados_meteorologicos_2024.parquet").repartition(12))
df = df.withColumn("hora_utc", col("hora_utc").cast("int"))
df = df.withColumnRenamed("estacao", "cidade")
df = df.drop(df.altitude)

In [7]:
df

data,hora_utc,cidade,uf,regiao,precipitacao_total_horario_mm_,pressao_atmosferica_ao_nivel_da_estacao_horaria_mb_,pressao_atmosferica_max_na_hora_ant_aut_mb_,pressao_atmosferica_min_na_hora_ant_aut_mb_,temperatura_do_ar_bulbo_seco_horaria_degc_,temperatura_do_ponto_de_orvalho_degc_,temperatura_maxima_na_hora_ant_aut_degc_,temperatura_minima_na_hora_ant_aut_degc_,temperatura_orvalho_max_na_hora_ant_aut_degc_,temperatura_orvalho_min_na_hora_ant_aut_degc_,umidade_rel_max_na_hora_ant_aut_pcnt_,umidade_rel_min_na_hora_ant_aut_pcnt_,umidade_relativa_do_ar_horaria_pcnt_,vento_direcao_horaria_gr_deg_gr_,vento_rajada_maxima_ms_,vento_velocidade_horaria_ms_
2024-01-16,2,JUIZ DE FORA,MG,SE,0.0,912.1,912.3,912.0,22.2,17.6,23.3,22.2,18.4,17.5,79.0,70.0,75.0,296.0,8.3,3.8
2024-02-04,13,LUZIANIA,GO,CO,NULL,906.1,906.3,906.1,24.1,15.3,24.4,22.0,18.1,15.3,76.0,58.0,58.0,85.0,5.2,3.0
2024-01-09,9,CAPELINHA,MG,SE,0.0,911.6,911.6,911.1,19.1,18.4,19.1,18.7,18.4,18.0,96.0,96.0,96.0,112.0,0.0,0.0
2024-01-19,17,PARNAIBA,PI,NE,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,40.0,9.2,5.0
2024-02-16,0,ARAGARCAS,GO,CO,0.0,976.2,976.2,975.3,NULL,NULL,NULL,NULL,NULL,NULL,79.0,76.0,79.0,241.0,3.6,0.8
2024-01-30,9,ITAPACI,GO,CO,NULL,951.9,952.0,951.3,21.4,20.9,21.4,21.1,20.9,20.6,97.0,97.0,97.0,324.0,1.0,0.3
2024-02-01,8,CAMPOS NOVOS,SC,S,0.0,904.6,904.6,904.1,17.3,16.9,17.9,17.3,17.3,16.8,97.0,96.0,97.0,99.0,4.6,3.3
2024-02-06,18,BATAGUASSU,MS,CO,10.4,968.6,969.9,968.6,30.2,21.0,30.2,23.1,24.7,18.4,100.0,57.0,58.0,146.0,12.3,1.1
2024-01-09,2,GOVERNADOR VALADARES,MG,SE,0.0,990.9,991.1,990.9,23.7,21.7,23.8,23.7,21.8,21.7,89.0,88.0,89.0,96.0,3.2,1.1
2024-03-07,11,SOBRAL,CE,NE,0.0,1002.5,1002.6,1001.8,25.3,21.7,25.3,24.6,21.7,21.2,81.0,80.0,80.0,33.0,0.0,0.0


In [8]:
# Renomeando as colunas para aeroportos de origem
df_origem = df.select(
    *[col(c).alias(c + '_origem') for c in df.columns])

# Renomeando as colunas para aeroportos de destino
df_destino = df.select(
    *[col(c).alias(c + '_destino') for c in df.columns])

df_origem = df_origem.withColumnRenamed("data_origem", "data_partida") \
                 .withColumnRenamed("hora_utc_origem", "hora_partida")

df_destino = df_destino.withColumnRenamed("data_destino", "data_chegada")\
                 .withColumnRenamed("hora_utc_destino", "hora_chegada")

## Df voos

In [9]:
from pyspark.sql import functions as F
from pyspark.sql.functions import to_date, col

In [11]:
df_voos=spark.read.option("header", "true").csv("dados_tratados/historico_voo_tratados_2024.csv").repartition(10)
df_voos= df_voos.filter((df_voos.pais_origem == 'BRASIL') & (df_voos.pais_destino == 'BRASIL'))
df_voos = df_voos.drop(df_voos.latgeopoint_destino, df_voos.longeopoint_destino,df_voos.latgeopoint_origem, df_voos.longeopoint_origem)

In [12]:
#[row.cidade_destino for row in df_voos.select("cidade_destino").distinct().collect()]
#[row.cidade_origem for row in df_voos.select("cidade_origem").distinct().collect()]

In [13]:
df_voos = df_voos.withColumn('partida_prevista_data', to_date(col('partida_prevista')))
df_voos = df_voos.withColumn('chegada_prevista_data', to_date(col('chegada_prevista')))

# Converter hora_partida e hora_chegada em df_voos para integer
df_voos = df_voos.withColumn("hora_partida", F.col("hora_partida").cast("integer")) \
                 .withColumn("hora_chegada", F.col("hora_chegada").cast("integer"))

In [14]:
# Defina o mapeamento de para->destino como dicionário
mapping_origem = {
    ("SENA MADUREIRA", "AC"): ("RIO BRANCO", "AC"),
    ("TABATINGA", "AM"): ("CRUZEIRO DO SUL", "AC"),
    ("BONITO", "PA"): ("CASTANHAL", "PA"),
    ("PARAUAPEBAS", "PA"): ("XINGUARA", "PA"),
    ("CASCAVEL", "CE"): ("FORTALEZA", "CE"),
    ("JUAZEIRO DO NORTE", "CE"): ("BARBALHA", "CE"),
    ("PARNAMIRIM", "RN"): ("NATAL", "RN"),
    ("BAYEUX", "PB"): ("CONDE", "PB"),
    ("BONITO", "PE"): ("PALMARES", "PE"),
    ("FERNANDO DE NORONHA", "PE"): ("NATAL", "RN"),
    ("GOIANA", "PE"): ("CONDE", "PB"),
    ("RIO LARGO", "AL"): ("CORURIPE", "AL"),
    ("PAULO AFONSO", "BA"): ("PIRANHAS", "AL"),
    ("CONFINS", "MG"): ("SETE LAGOAS", "MG"),
    ("TOLEDO", "MG"): ("ITAPEVA", "MG"),
    ("CABO FRIO", "RJ"): ("ARRAIAL DO CABO", "RJ"),
    ("AREALVA", "SP"): ("IBITINGA", "SP"),
    ("CAMPINAS", "SP"): ("ITAPIRA", "SP"),
    ("GUARULHOS", "SP"): ("BARUERI", "SP"),
    ("PIRASSUNUNGA", "SP"): ("CASA BRANCA", "SP"),
    ("CASCAVEL", "PR"): ("IGUATU", "PR"),
    ("LONDRINA", "PR"): ("FLORESTA", "PR"),
    ("TOLEDO", "PR"): ("IGUATU", "PR"),
    ("JOINVILLE", "SC"): ("RIO NEGRINHO", "SC"),
    ("NAVEGANTES", "SC"): ("INDAIAL", "SC"),
    ("CAXIAS DO SUL", "RS"): ("CANELA", "RS"),
    ("PELOTAS", "RS"): ("RIO GRANDE", "RS"),
    ("CALDAS NOVAS", "GO"): ("MORRINHOS", "GO"),
}

# Função auxiliar para criar o case when para cidade_origem e uf_origem
def case_when_replace(mapping, col_city, col_state):
    case_expr = F.when(F.lit(False), F.lit(None))  # Inicializa com uma condição que nunca é verdadeira
    for (city, state), (new_city, new_state) in mapping.items():
        case_expr = case_expr.when(
            (F.col(col_city) == city) & (F.col(col_state) == state),
            F.struct(F.lit(new_city).alias("new_city"), F.lit(new_state).alias("new_state"))
        )
    return case_expr.otherwise(F.struct(F.col(col_city).alias("new_city"), F.col(col_state).alias("new_state")))

# Aplicando a função para criar as colunas cidade_origem_join e uf_origem_join
df_voos = df_voos.withColumn(
    "origem_replacement",
    case_when_replace(mapping_origem, "cidade_origem", "uf_origem_x")
)

# Extraindo as colunas cidade_origem_join e uf_origem_join a partir da coluna de struct
df_voos = df_voos.withColumn("cidade_origem_join", F.col("origem_replacement.new_city")) \
                 .withColumn("uf_origem_join", F.col("origem_replacement.new_state")) \
                 .drop("origem_replacement")

# Aplicando o mesmo procedimento para cidade_destino e uf_destino_x, com mapeamento similar, se necessário
# Exemplo:
df_voos = df_voos.withColumn(
    "destino_replacement",
    case_when_replace(mapping_origem, "cidade_destino", "uf_destino_x")
)

df_voos = df_voos.withColumn("cidade_destino_join", F.col("destino_replacement.new_city")) \
                 .withColumn("uf_destino_join", F.col("destino_replacement.new_state")) \
                 .drop("destino_replacement")


In [15]:
# Renomeie as colunas após remover as duplicatas
df_voos = df_voos.withColumnRenamed("partida_prevista_data", "data_partida") \
                 .withColumnRenamed("hora_partida", "hora_partida") \
                 .withColumnRenamed("cidade_origem", "cidade_origem") \
                 .withColumnRenamed("uf_origem_x", "uf_origem") \
                 .withColumnRenamed("chegada_prevista_data", "data_chegada") \
                 .withColumnRenamed("uf_destino_x", "uf_destino")

# Converta para string, se necessário
df_voos = df_voos.withColumn("data_partida", col("data_partida").cast("string")) \
                 .withColumn("data_chegada", col("data_chegada").cast("string"))

## Join entre os datasets

In [16]:
from pyspark.sql import functions as F

# Realizar o join entre df_voos e df_origem usando as colunas de partida
df_voos_origem = df_voos.join(
    df_origem,
    on=[
        df_voos["data_partida"] == df_origem["data_partida"],
        df_voos["hora_partida"] == df_origem["hora_partida"],
        df_voos["cidade_origem_join"] == df_origem["cidade_origem"],
        df_voos["uf_origem_join"] == df_origem["uf_origem"]
    ],
    how="inner"
).drop(
    df_origem["data_partida"],
    df_origem["hora_partida"],
    df_origem["cidade_origem"],
    df_origem["uf_origem"],
    df_voos["cidade_origem_join"],
    df_voos["uf_origem_join"]
)

# Realizar o join entre o resultado anterior (df_voos_origem) e df_destino usando as colunas de chegada
df_voos_completo = df_voos_origem.join(
    df_destino,
    on=[
        df_voos_origem["data_chegada"] == df_destino["data_chegada"],
        df_voos_origem["hora_chegada"] == df_destino["hora_chegada"],
        df_voos_origem["cidade_destino_join"] == df_destino["cidade_destino"],
        df_voos_origem["uf_destino_join"] == df_destino["uf_destino"]
    ],
    how="inner"
).drop(
    df_destino["data_chegada"],
    df_destino["hora_chegada"],
    df_destino["cidade_destino"],
    df_destino["uf_destino"],
    df_voos_origem["uf_destino_join"],
    df_voos_origem["cidade_destino_join"])

# Verificar o esquema final após o join
df_voos_completo.printSchema()

root
 |-- nome_empresas: string (nullable = true)
 |-- numero_voo: string (nullable = true)
 |-- codigo_di: string (nullable = true)
 |-- codigo_tipo_linha: string (nullable = true)
 |-- partida_prevista: string (nullable = true)
 |-- partida_real: string (nullable = true)
 |-- chegada_prevista: string (nullable = true)
 |-- chegada_real: string (nullable = true)
 |-- situacao_voo: string (nullable = true)
 |-- descricao_origem: string (nullable = true)
 |-- pais_origem: string (nullable = true)
 |-- continente_origem: string (nullable = true)
 |-- descricao_destino: string (nullable = true)
 |-- pais_destino: string (nullable = true)
 |-- continente_destino: string (nullable = true)
 |-- status_do_voo: string (nullable = true)
 |-- delta_tempo_partida_delta: string (nullable = true)
 |-- delta_tempo_chegada_delta: string (nullable = true)
 |-- delta_tempo_partida_int: string (nullable = true)
 |-- delta_tempo_chegada_int: string (nullable = true)
 |-- cidade_origem: string (nullable =

In [17]:
colunas = df_voos_completo.columns
[coluna for coluna in set(colunas) if colunas.count(coluna) > 1]

[]

In [18]:
# Identificar colunas duplicadas
colunas = df_voos_completo.columns
duplicadas = [coluna for coluna in set(colunas) if colunas.count(coluna) > 1]

print("Colunas duplicadas:", duplicadas)

# Remover duplicatas
df_voos_completo = df_voos_completo.drop(*duplicadas)
df_voos_completo.printSchema()

Colunas duplicadas: []
root
 |-- nome_empresas: string (nullable = true)
 |-- numero_voo: string (nullable = true)
 |-- codigo_di: string (nullable = true)
 |-- codigo_tipo_linha: string (nullable = true)
 |-- partida_prevista: string (nullable = true)
 |-- partida_real: string (nullable = true)
 |-- chegada_prevista: string (nullable = true)
 |-- chegada_real: string (nullable = true)
 |-- situacao_voo: string (nullable = true)
 |-- descricao_origem: string (nullable = true)
 |-- pais_origem: string (nullable = true)
 |-- continente_origem: string (nullable = true)
 |-- descricao_destino: string (nullable = true)
 |-- pais_destino: string (nullable = true)
 |-- continente_destino: string (nullable = true)
 |-- status_do_voo: string (nullable = true)
 |-- delta_tempo_partida_delta: string (nullable = true)
 |-- delta_tempo_chegada_delta: string (nullable = true)
 |-- delta_tempo_partida_int: string (nullable = true)
 |-- delta_tempo_chegada_int: string (nullable = true)
 |-- cidade_ori

In [19]:
def missing_values_table_spark(df):
    """
    Cria uma tabela resumindo a quantidade e a porcentagem de valores ausentes em cada coluna do DataFrame PySpark.

    Args:
    df (spark.DataFrame): DataFrame para análise de valores ausentes.

    Returns:
    spark.DataFrame: Uma tabela com o número e a porcentagem de valores ausentes por coluna.
    """
    # Calcula o número total de linhas no DataFrame
    total_rows = df.count()

    # Calcula o número total de valores ausentes por coluna
    missing_count = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

    # Calcula a porcentagem de valores ausentes por coluna
    missing_percent = df.select([(count(when(col(c).isNull(), c)) / total_rows * 100).alias(c) for c in df.columns])

    # Preparando para juntar contagens e percentagens
    missing_count = missing_count.withColumnRenamed(missing_count.columns[0], 'Missing Values')
    missing_percent = missing_percent.withColumnRenamed(missing_percent.columns[0], '% of Total Values')

    # Junta as contagens e as percentagens em um DataFrame
    missing_table = missing_count.join(missing_percent)

    # Ordena as colunas com valores ausentes por porcentagem de forma decrescente
    missing_table = missing_table.orderBy(col('% of Total Values').desc())

    # Imprime um resumo das colunas com valores ausentes
    print("Your selected dataframe has " + str(len(df.columns)) + " columns.\n"      
          "There are " + str(missing_table.count()) + " columns that have missing values.")

    return missing_table

In [20]:
missing_values_table_spark(df_voos_completo)

Your selected dataframe has 75 columns.
There are 1 columns that have missing values.


Missing Values,numero_voo,codigo_di,codigo_tipo_linha,partida_prevista,partida_real,chegada_prevista,chegada_real,situacao_voo,descricao_origem,pais_origem,continente_origem,descricao_destino,pais_destino,continente_destino,status_do_voo,delta_tempo_partida_delta,delta_tempo_chegada_delta,delta_tempo_partida_int,delta_tempo_chegada_int,cidade_origem,uf_origem,cidade_destino,uf_destino,mes_partida,dia_semana_partida,dia_mes_partida,hora_partida,mes_chegada,dia_semana_chegada,dia_mes_chegada,hora_chegada,altitude_origem,latitude_aero_origem,longitude_aero_origem,altitude_destino,latitude_aero_destino,longitude_aero_destino,rota,data_partida,data_chegada,regiao_origem,precipitacao_total_horario_mm__origem,pressao_atmosferica_ao_nivel_da_estacao_horaria_mb__origem,pressao_atmosferica_max_na_hora_ant_aut_mb__origem,pressao_atmosferica_min_na_hora_ant_aut_mb__origem,temperatura_do_ar_bulbo_seco_horaria_degc__origem,temperatura_do_ponto_de_orvalho_degc__origem,temperatura_maxima_na_hora_ant_aut_degc__origem,temperatura_minima_na_hora_ant_aut_degc__origem,temperatura_orvalho_max_na_hora_ant_aut_degc__origem,temperatura_orvalho_min_na_hora_ant_aut_degc__origem,umidade_rel_max_na_hora_ant_aut_pcnt__origem,umidade_rel_min_na_hora_ant_aut_pcnt__origem,umidade_relativa_do_ar_horaria_pcnt__origem,vento_direcao_horaria_gr_deg_gr__origem,vento_rajada_maxima_ms__origem,vento_velocidade_horaria_ms__origem,regiao_destino,precipitacao_total_horario_mm__destino,pressao_atmosferica_ao_nivel_da_estacao_horaria_mb__destino,pressao_atmosferica_max_na_hora_ant_aut_mb__destino,pressao_atmosferica_min_na_hora_ant_aut_mb__destino,temperatura_do_ar_bulbo_seco_horaria_degc__destino,temperatura_do_ponto_de_orvalho_degc__destino,temperatura_maxima_na_hora_ant_aut_degc__destino,temperatura_minima_na_hora_ant_aut_degc__destino,temperatura_orvalho_max_na_hora_ant_aut_degc__destino,temperatura_orvalho_min_na_hora_ant_aut_degc__destino,umidade_rel_max_na_hora_ant_aut_pcnt__destino,umidade_rel_min_na_hora_ant_aut_pcnt__destino,umidade_relativa_do_ar_horaria_pcnt__destino,vento_direcao_horaria_gr_deg_gr__destino,vento_rajada_maxima_ms__destino,vento_velocidade_horaria_ms__destino,% of Total Values,numero_voo,codigo_di,codigo_tipo_linha,partida_prevista,partida_real,chegada_prevista,chegada_real,situacao_voo,descricao_origem,pais_origem,continente_origem,descricao_destino,pais_destino,continente_destino,status_do_voo,delta_tempo_partida_delta,delta_tempo_chegada_delta,delta_tempo_partida_int,delta_tempo_chegada_int,cidade_origem,uf_origem,cidade_destino,uf_destino,mes_partida,dia_semana_partida,dia_mes_partida,hora_partida,mes_chegada,dia_semana_chegada,dia_mes_chegada,hora_chegada,altitude_origem,latitude_aero_origem,longitude_aero_origem,altitude_destino,latitude_aero_destino,longitude_aero_destino,rota,data_partida,data_chegada,regiao_origem,precipitacao_total_horario_mm__origem,pressao_atmosferica_ao_nivel_da_estacao_horaria_mb__origem,pressao_atmosferica_max_na_hora_ant_aut_mb__origem,pressao_atmosferica_min_na_hora_ant_aut_mb__origem,temperatura_do_ar_bulbo_seco_horaria_degc__origem,temperatura_do_ponto_de_orvalho_degc__origem,temperatura_maxima_na_hora_ant_aut_degc__origem,temperatura_minima_na_hora_ant_aut_degc__origem,temperatura_orvalho_max_na_hora_ant_aut_degc__origem,temperatura_orvalho_min_na_hora_ant_aut_degc__origem,umidade_rel_max_na_hora_ant_aut_pcnt__origem,umidade_rel_min_na_hora_ant_aut_pcnt__origem,umidade_relativa_do_ar_horaria_pcnt__origem,vento_direcao_horaria_gr_deg_gr__origem,vento_rajada_maxima_ms__origem,vento_velocidade_horaria_ms__origem,regiao_destino,precipitacao_total_horario_mm__destino,pressao_atmosferica_ao_nivel_da_estacao_horaria_mb__destino,pressao_atmosferica_max_na_hora_ant_aut_mb__destino,pressao_atmosferica_min_na_hora_ant_aut_mb__destino,temperatura_do_ar_bulbo_seco_horaria_degc__destino,temperatura_do_ponto_de_orvalho_degc__destino,temperatura_maxima_na_hora_ant_aut_degc__destino,temperatura_minima_na_hora_ant_aut_degc__de

# Salvando o dataset tratado com dados de meterologia

In [21]:
df_voos_completo.count()

532588

In [22]:
df_voos_completo.dropna(thresh=3).count()

532588

In [23]:
# Converter o DataFrame Spark para Pandas
df_voos_completo_pandas = df_voos_completo.toPandas()

# Salvar em CSV
df_voos_completo_pandas.to_csv("df_voos_completo_2024.csv", index=False)

In [11]:
# Reparticionando o DataFrame para uma única partição
df_joined_final = df_joined_final.repartition(1)

# Salvando o DataFrame em um único arquivo CSV
(df_joined_final.write
     .option("header", "true")
     .mode("overwrite")
     .csv("dados_tratados/historico_voo_meteorologia.csv"))

# Buscar por cidades próximas

In [19]:


file_path = 'C:/Users/pedro/Documents/Curso de pos graduação de EST/DADOS_CNPJ/df_mun_cod_lat_long.csv'

df_municipios = pd.read_csv(file_path)

# Remover caracteres especiais da coluna NOME_MUNICIPIO, tratando NaNs
df_municipios["NOME_MUNICIPIO"] = df_municipios["NOME_MUNICIPIO"].apply(
    lambda x: re.sub(r"[^a-zA-Z0-9\s]", "", str(x)) if pd.notnull(x) else ""
)

In [16]:
airports =  { 'GOIANIA', 'BRASILIA', 'BELEM', 'AREALVA', 'SAO LUIS', 'ITAITUBA', 'PARAUAPEBAS', 'BAYEUX',
    'SAO JOSE DOS CAMPOS', 'SAO GABRIEL DA CACHOEIRA', 'SOROCABA', 'BOA VISTA', 'RIO DE JANEIRO',
    'UNA', 'PRESIDENTE PRUDENTE', 'GUARULHOS', 'CAMPOS DOS GOYTACAZES', 'MARINGA', 'CABO FRIO',
    'SAO JOSE DO RIO PRETO', 'UBERABA', 'PARNAIBA', 'SINOP', 'FERNANDO DE NORONHA', 'URUGUAIANA',
    'GOIANA', 'PARNAMIRIM', 'CAMPINAS', 'CONFINS', 'RECIFE', 'ILHEUS', 'SAO CARLOS', 'SANTO ANGELO',
    'PAULO AFONSO', 'CRUZEIRO DO SUL', 'ALTAMIRA', 'RIO VERDE', 'ORIXIMINA', 'SAO JOSE DOS PINHAIS',
    'FORTALEZA', 'CAMPO GRANDE', 'UBERLANDIA', 'RONDONOPOLIS', 'TRES LAGOAS', 'PARINTINS', 'JI-PARANA',
    'PALMAS', 'TEFE', 'GOVERNADOR VALADARES', 'MANAUS', 'NAVEGANTES', 'SANTAREM', 'VILHENA',
    'ALTA FLORESTA', 'TOLEDO', 'ARACAJU', 'JUAZEIRO DO NORTE', 'SANTANA DO PARAISO', 'MACAE',
    'CALDAS NOVAS', 'FLORIANOPOLIS', 'PORTO SEGURO', 'FOZ DO IGUACU', 'SENA MADUREIRA', 'MARILIA',
    'PASSO FUNDO', 'CAMPINA GRANDE', 'SAO GONCALO DO AMARANTE', 'MARABA', 'CHAPECO', 'ARACATUBA',
    'BONITO', 'VARZEA GRANDE', 'PORTO ALEGRE', 'BELO HORIZONTE', 'CASCAVEL', 'CORUMBA', 'IMPERATRIZ',
    'SALVADOR', 'CAXIAS DO SUL', 'PORTO VELHO', 'JOINVILLE', 'LONDRINA', 'BARREIRAS', 'ARAXA',
    'CACOAL', 'VITORIA', 'RIBEIRAO PRETO', 'PETROLINA', 'EIRUNEPE', 'LENCOIS', 'MACAPA', 'MOSSORO',
    'TABATINGA', 'PONTA PORA', 'TERESINA', 'SANTA MARIA', 'GUANAMBI', 'PATOS DE MINAS', 'SAO PAULO',
    'RIO LARGO', 'PELOTAS', 'MONTES CLAROS', 'PIRASSUNUNGA'}

meteorology_cities = {
    "BARRETOS", "CAMARATUBA", "FREDERICO WESTPHALEN", "GOIANIA", "NOVA FATIMA", "JAGUARUANA", "BATAGUASSU", "TUPA",
    "SAO MATEUS", "ITAPETINGA", "POSSE", "VARGINHA", "AMAMBAI", "AQUIDAUANA", "ITAPIRA", "RIO SONO", "ARACUAI",
    "JOAO PINHEIRO", "CAMPINA DA LAGOA", "JOACABA", "MORADA NOVA", "RESENDE", "CANTO DO BURITI", "BARBALHA",
    "SANTIAGO", "BELEM", "ITIQUIRA", "NITEROI", "PARATY", "LAGES", "POCO VERDE", "TEUTONIA", "XINGUARA",
    "GUIRATINGA", "SAO JOAQUIM", "BRASILIA", "BARRA DO CORDA", "LARANJEIRAS DO SUL", "LAGOA VERMELHA", "RONDON DO PARA",
    "SAO BORJA", "ALTO TAQUARI", "SAO MATEUS DO SUL", "QUARAI", "ITAITUBA", "VALENCA DO PIAUI", "AGUAS EMENDADAS",
    "SAO SIMAO", "SALINOPOLIS", "CAMPO NOVO DOS PARECIS", "IBIMIRIM", "PRESIDENTE FIGUEIREDO", "DOM PEDRITO",
    "FEIJO", "TIANGUA", "ITAPACI", "CAPELINHA", "MACEIO", "TERESOPOLIS", "ITAOBIM", "EDEIA", "FRANCA",
    "PALMARES", "ESTREITO", "AREIA", "SAO LUIS", "JAGUARAO", "TRES RIOS", "SANTO ANTONIO DO LESTE", "APIACAS",
    "PASSA QUATRO", "CAMPOS NOVOS", "SAO RAIMUNDO NONATO", "CONDE", "CRUZ ALTA", "CAMPOS DO JORDAO", "URUSSANGA",
    "FLORESTAL", "MAJOR VIEIRA", "SOROCABA", "OBIDOS", "DIAMANTINA", "PIRIPIRI", "BOA VISTA", "ITUVERAVA",
    "CAMPO MAIOR", "IPANGUACU", "CURITIBA", "PRESIDENTE PRUDENTE", "SANTA CRUZ", "UNA", "RIO GRANDE",
    "COLINAS DO TOCANTINS", "SANTO AUGUSTO", "QUIXERAMOBIM", "CHAPADINHA", "RIBEIRA DO AMPARO", "SAO GABRIEL DO OESTE",
    "SAO SEBASTIAO DO PARAISO", "IGUATU", "APUI", "RIO DE JANEIRO", "CASTELO DO PIAUI", "PICO DO COUTO", "NOVA FRIBURGO",
    "PATOS", "CRATEUS", "JANUARIA", "IBITINGA", "OURICURI", "SAO LUIS DO QUITUNDE", "COLINAS", "CAMPOS LINDOS",
    "CAPAO DO LEAO", "CAMPOS DOS GOYTACAZES", "TRES MARIAS", "PLACAS", "SAO MIGUEL DO ARAGUAIA", "CABROBO", "CAIAPONIA",
    "AGUAS VERMELHAS", "ITUPORANGA", "PARANAPOEMA", "SETE LAGOAS", "ERECHIM", "ITIRUCU", "UBERABA", "COTRIGUACU",
    "POMPEU", "CAPITAO POCO", "QUEIMADAS", "CARATINGA", "CAMBARA DO SUL", "VENTANIA", "JEREMOABO", "SANTA MARIA DAS BARREIRAS",
    "MARIA DA FE", "FORMIGA", "NOSSA SENHORA DA GLORIA", "PARNAIBA", "ABROLHOS", "SINOP", "VALENCA", "ITAUBAL",
    "CARUARU", "MARINGA", "COARI", "S.J. DO RIO CLARO", "EUCLIDES DA CUNHA", "BRAZLANDIA", "ITABERABA", "BRAGANCA PAULISTA",
    "TUCURUI", "URUGUAIANA", "RIO BRANCO", "ANGICAL DO PIAUI", "SALTO DO CEU", "Laguna", "MARILANDIA", "AIMORES",
    "ALMENARA", "PALMEIRA DOS INDIOS", "CUIABA", "JACOBINA", "PARANA", "ARAPIRACA", "DOIS VIZINHOS", "SONORA",
    "PAO DE ACUCAR", "FORMOSO DO ARAGUAIA", "RIO CLARO", "SAO MIGUEL ARCANJO", "DORES DO INDAIA", "AGUA BOA",
    "SANTA TERESA", "SORRISO", "Paty do Alferes", "BOM JARDIM DA SERRA", "SACRAMENTO", "S. G. DA CACHOEIRA",
    "PORANGATU", "CAMETA", "IPIAU", "ILHEUS", "BURITIS", "BEBEDOURO", "NATAL", "JUARA", "LINHARES", "MOCAMBINHO",
    "ITAPORANGA", "VITORIA DA CONQUISTA", "AVARE", "CASTRO", "SAO JOSE DOS AUSENTES", "MATEIROS", "CRUZEIRO DO SUL",
    "CAMAQUA", "ALTAMIRA", "JUIZ DE FORA", "RECIFE", "SAO CARLOS", "RIO VERDE",    'LINS', 'JOSE BONIFACIO', 'TUPANCIRETA', 
    'PIUM', 'VACARIA', 'ARAGARCAS', 'SOBRAL', 'MAUES', 'UBERLANDIA', 'BREVES', 
    'PICOS', 'CAMPO GRANDE', 'SAO JOAO DEL REI', 'RIO NEGRINHO', 'FORTALEZA', 'JOAO PESSOA', 'NOVA UBIRATA', 
    'ITUIUTABA', 'CALCANHAR', 'ALTO PARNAIBA', 'REDENCAO', 'ESPINOSA', 'BAMBUI', 'BARCELOS', 'JALES', 'RONDONOPOLIS', 
    'TUCUMA', 'SANTANA DO ARAGUAIA', 'APODI', 'REMANSO', 'CORURIPE', 'VENDA NOVA DO IMIGRANTE', 'RANCHO QUEIMADO', 
    'BAGE', 'PORTO ESTRELA', 'MONTE ALEGRE DE GOIAS', 'SAO MIGUEL DO OESTE', 'CAICO', 'OIAPOQUE', 'PREGUICAS', 
    'EPITACIOLANDIA', 'BARRA BONITA', 'FEIRA DE SANTANA', 'SAO GONCALO', 'OURO BRANCO', 'QUERENCIA', 'SAO JOAO DO PIAUI', 
    'MACAU', 'PLANALTO', 'CAMPO VERDE', 'PORTO DE MOZ', 'IBIRITE', 'GURUPI', 'JUTI', 'TRES LAGOAS', 'ITABAIANA', 
    'MONTE ALEGRE', 'BREJO GRANDE', 'PARINTINS', 'MEDICILANDIA', 'UNAI', 'TAUA', 'MANACAPURU', 'VILA BELA DA SANTISSIMA TRINDADE', 
    'IBOTIRAMA', 'URUCUI', 'GOIOERE', 'LUZIANIA', 'SANTA ROSA', 'CAMPINA VERDE', 'ARAGUAINA', 'JATAI', 'COSTA RICA', 
    'ECOPORANGA', 'VALPARAISO', 'CHAPADAO DO SUL', 'GENERAL CARNEIRO', 'CASA BRANCA', 'CLEVELANDIA', 'ITAQUIRAI', 
    'OURINHOS', 'ALMAS', 'PIRES DO RIO', 'CANELA', 'PARAUNA', 'BOCA DO ACRE', 'DUQUE DE CAXIAS', 'PALMAS', 
    'GOVERNADOR VALADARES', 'SAO VICENTE DO SUL', 'ITAPEVA', 'NOVA MARINGA', 'MANAUS', 'CIDADE GAUCHA', 'ALFREDO CHAVES', 
    'CORONEL PACHECO', 'DIAMANTE DO NORTE', 'ARRAIAL DO CABO', 'SERAFINA CORREA', 'MONTE VERDE', 'VILHENA', 'SOURE', 
    'ALTA FLORESTA', 'JARDIM', 'SERRA DOS AIMORES', 'DELFINO', 'JOAQUIM TAVORA', 'NOVA PORTEIRINHA', 'CARAVELAS', 
    'PORTO GRANDE', 'SANTAREM', 'PEIXE', 'BACABAL', 'AUTAZES', 'CURACA', 'BRAGANCA', 'CRISTALINA', 'ITAPIPOCA', 
    'NOVO REPARTIMENTO', 'CACAPAVA DO SUL', 'JAGUARIBE', 'SAQUAREMA', 'ZE DOCA', 'MARACAJU', 'CASTANHAL', 'CALDAS', 
    'MANTENA', 'TOME ACU', 'CARACOL', 'SAPEZAL', 'ALTO PARAISO DE GOIAS', 'CASSILANDIA', 'COLOMBO', 'GILBUES', 
    'ITAMARAJU', 'CAXIAS', 'ARACAJU', 'PARANOA', 'DIANOPOLIS', 'MACAE', 'QUIXADA', 'AMARGOSA', 'MURIAE', 'IGUAPE', 
    'CARLINDA', 'ALTO ARAGUAIA', 'VILA VELHA', 'BERTIOGA', 'RIO BRILHANTE', 'INDAIAL', 'PATROCINIO', 'PIRACICABA', 
    'PRIMAVERA DO LESTE', 'OEIRAS', 'ARIRANHA', 'SOLEDADE', 'ICARAIMA', 'AGUA CLARA', 'FLORIANO', 'SERRINHA', 
    'MOSTARDAS', 'MARILIA', 'SILVA JARDIM', 'PORTO SEGURO', 'MINEIROS', 'ITACOATIARA', 'TRAMANDAI', 'CARMO', 
    'CATALAO', 'FLORIANOPOLIS', 'FOZ DO IGUACU', 'PEDRO AFONSO', 'REGISTRO', 'SAO JOSE DO XINGU', 'DOURADOS', 
    'SALGUEIRO', 'FORMOSA DO RIO PRETO', 'SALINAS', 'MORRETES', 'ITUMBIARA', 'GUANHAES', 'ARIQUEMES', 'BARUERI', 
    'RIO DO CAMPO', 'CHAPECO', 'HUMAITA', 'VICOSA', 'PARACATU', 'PARAGOMINAS', 'IPORA', 'CAMPINA GRANDE', 
    'AFONSO CLAUDIO', 'ITAPOA', 'JAPIRA', 'BURITIRAMA', 'DRACENA', 'PASSOS', 'BAIXA GRANDE DO RIBEIRO', 'TURIACU', 
    'IVAI', 'MARABA', 'BURITICUPU', 'PASSO FUNDO', 'CORRENTE', 'RANCHARIA', 'COXIM', 'ENCRUZILHADA DO SUL', 
    'RIO PARDO', 'ARAGUATINS', 'BELO HORIZONTE', 'ITABAIANINHA', 'TORRES', 'NOVA TEBAS', 'ROSARIO OESTE', 
    'PORTO ALEGRE', 'IRECE', 'LABREA', 'MARAU', 'MARIANOPOLIS DO TO', 'ARAGUACU', 'MACAJUBA', 'PIRAPORA', 'PIRANHAS', 
    'NOVO ARIPUANA', 'BALSAS', 'CORUMBA', 'MACHADO', 'IMPERATRIZ', 'BRUMADO', 'OLIVEIRA', 'PORTO MURTINHO', 
    'SURUBIM', 'SETE QUEDAS', 'PARQUE ESTADUAL CHANDLESS', 'BENTO GONCALVES', 'GUARANTA DO NORTE', 'XANXERE', 
    'FLORESTA', 'SAO GABRIEL', 'BOM JESUS DO PIAUI', 'SANTA ROSA DO TOCANTINS', 'PAULISTANA', 'RIO PARDO DE MINAS', 
    'PORTO VELHO', 'BARREIRAS', 'ALEGRETE', 'SAO LUIZ GONZAGA', 'RIO URUBU', 'TAUBATE', 'MONTEIRO', 'SAO ROMAO', 
    'ARAXA', 'GAUCHA DO NORTE', 'TANGARA DA SERRA', 'URUCARA', 'SANTA RITA DE CASSIA', 'MORRINHOS', 'PACAJA', 
    'GARANHUNS', 'SALVADOR', 'BARBACENA', 'DIVINOPOLIS', 'MIRANDA', 'CACOAL', 'MARECHAL THAUMATURGO', 'BAURU', 
    'PORTO WALTER', 'EIRUNEPE', 'SEROPEDICA', 'CURVELO', 'DIONISIO CERQUEIRA', 'SILVANIA', 'CHAPADA GAUCHA', 
    'CACADOR', 'GRAJAU', 'CAMPOS SALES', 'CONCEICAO DO ARAGUAIA', 'GOIAS', 'PRESIDENTE KENNEDY', 'SERRA DOS CARAJAS', 
    'PALMEIRA DAS MISSOES', 'PETROLINA', 'MANHUACU', 'TIMOTEO', 'LENCOIS', 'IVINHEMA', 'FAROL de SANTANA', 
    'ANGRA DOS REIS', 'PRADOPOLIS', 'CORRENTINA', 'VITORIA', 'BARRA', 'CAMBUCI', 'PIATA', 'ARARANGUA', 
    'Santa Vitoria do Palmar', 'MOSSORO', 'COMODORO', 'CACHOEIRA PAULISTA', 'CAMPO BOM', 'PARANAIBA', 
    'BOM JESUS DA LAPA', 'MINA DO PALITO', 'SANTA FE DO ARAGUAIA', 'DOM ELISEU', 'CONCEICAO DAS ALAGOAS', 
    'ALVORADA DO GURGUEIA', 'BARRA DO TURVO', 'PONTA PORA', 'MACAPA', 'JUINA', 'SENHOR DO BONFIM', 'GUARDA',
    'MONTALVANIA', 'TERESINA', 'ACARAU', 'CANGUCU', 'SANTANA DO LIVRAMENTO', 'SAO FELIX DO ARAGUAIA',
    'GUARAMIRANGA', 'SANTA MARIA', 'LUIZ EDUARDO MAGALHAES', 'NOVO HORIZONTE', 'BELMONTE', 'GUANAMBI',
    'PATOS DE MINAS', 'PARANATINGA', 'CAROLINA', 'SERRA NOVA DOURADA', 'SANTA MARIA MADALENA', 'GOIANESIA',
    'CABACEIRAS', 'SAO PAULO', 'PONTES E LACERDA', 'MAL. CANDIDO RONDON', 'SERRA TALHADA', 'BRASNORTE',
    'MANICORE', 'INACIO MARTINS', 'SIDROLANDIA', 'CURITIBANOS', 'TEOFILO OTONI', 'LAGOA DA CONFUSAO',
    'ITAJAI', 'ARCO VERDE', 'ALEGRE', 'NHUMIRIM', 'IBIRUBA', 'CRUZ DAS ALMAS', 'MONTES CLAROS', 'GAMA',
    'SAO LUIZ DO PARAITINGA', 'SAO FELIX DO XINGU'}

In [17]:
cidades_sem_dados_meteorologicos = airports - meteorology_cities
cidades_sem_dados_meteorologicos

{'ARACATUBA',
 'AREALVA',
 'BAYEUX',
 'BONITO',
 'CABO FRIO',
 'CALDAS NOVAS',
 'CAMPINAS',
 'CASCAVEL',
 'CAXIAS DO SUL',
 'CONFINS',
 'FERNANDO DE NORONHA',
 'GOIANA',
 'GUARULHOS',
 'JI-PARANA',
 'JOINVILLE',
 'JUAZEIRO DO NORTE',
 'LONDRINA',
 'NAVEGANTES',
 'ORIXIMINA',
 'PARAUAPEBAS',
 'PARNAMIRIM',
 'PAULO AFONSO',
 'PELOTAS',
 'PIRASSUNUNGA',
 'RIBEIRAO PRETO',
 'RIO LARGO',
 'SANTANA DO PARAISO',
 'SANTO ANGELO',
 'SAO GABRIEL DA CACHOEIRA',
 'SAO GONCALO DO AMARANTE',
 'SAO JOSE DO RIO PRETO',
 'SAO JOSE DOS CAMPOS',
 'SAO JOSE DOS PINHAIS',
 'SENA MADUREIRA',
 'TABATINGA',
 'TEFE',
 'TOLEDO',
 'VARZEA GRANDE'}

In [28]:

# Filtrar o DataFrame para as cidades especificadas e selecionar apenas as colunas relevantes
df_airport_cities = df_municipios[df_municipios['NOME_MUNICIPIO'].isin(cidades_sem_dados_meteorologicos)][['NOME_MUNICIPIO', 'LATITUDE', 'LONGITUDE', 'UF_SIGLA']]
df_airport_cities.reset_index(drop=True, inplace=True)

df_airports = df_airport_cities

# Filtrar o DataFrame para as cidades meteorológicas e selecionar apenas as colunas relevantes
df_meteorology = df_municipios[df_municipios['NOME_MUNICIPIO'].isin(meteorology_cities)][['NOME_MUNICIPIO', 'LATITUDE', 'LONGITUDE', 'UF_SIGLA']]
df_meteorology.reset_index(drop=True, inplace=True)

# Crie uma lista para armazenar os resultados
closest_cities = []

# Para cada cidade do dataset de aeroportos
for _, row_airport in df_airports.iterrows():
    airport_name = row_airport['NOME_MUNICIPIO']
    airport_coords = (row_airport['LATITUDE'], row_airport['LONGITUDE'])
    airport_uf = row_airport['UF_SIGLA']
    
    # Calcula a distância para todas as cidades meteorológicas
    df_meteorology['distance_to_airport'] = df_meteorology.apply(
        lambda row: geodesic((row['LATITUDE'], row['LONGITUDE']), airport_coords).kilometers, axis=1
    )
    
    # Identifique a cidade meteorológica mais próxima
    closest_city = df_meteorology.loc[df_meteorology['distance_to_airport'].idxmin()]
    
    # Adicione o resultado à lista
    closest_cities.append({
        'airport_city': airport_name,
        'airport_city_uf': airport_uf,
        'nearest_meteorology_city': closest_city['NOME_MUNICIPIO'],
        'nearest_meteorology_city_uf': closest_city['UF_SIGLA'],
        'distance_km': closest_city['distance_to_airport']
    })

# Converta os resultados em um DataFrame para visualização
df_closest_cities = pd.DataFrame(closest_cities)

# Exiba o DataFrame final com as cidades de aeroportos e suas cidades meteorológicas mais próximas
df_closest_cities

,airport_city,airport_city_uf,nearest_meteorology_city,nearest_meteorology_city_uf,distance_km
0,SENA MADUREIRA,AC,RIO BRANCO,AC,137.138983
1,TABATINGA,AM,CRUZEIRO DO SUL,AC,482.050904
2,BONITO,PA,CASTANHAL,PA,69.536848
3,PARAUAPEBAS,PA,XINGUARA,PA,113.866401
4,CASCAVEL,CE,FORTALEZA,CE,59.538374
5,JUAZEIRO DO NORTE,CE,BARBALHA,CE,10.784244
6,PARNAMIRIM,RN,NATAL,RN,17.652052
7,BAYEUX,PB,CONDE,PB,14.722712
8,BONITO,PE,PALMARES,PE,27.593541
9,FERNANDO DE NORONHA,PE,NATAL,RN,376.484214


In [38]:
df_closest_cities.distance_km.mean()

74.95914486809593

In [40]:
df_closest_cities[['airport_city',
 'airport_city_uf',
 'nearest_meteorology_city',
 'nearest_meteorology_city_uf',]]

,airport_city,airport_city_uf,nearest_meteorology_city,nearest_meteorology_city_uf
0,SENA MADUREIRA,AC,RIO BRANCO,AC
1,TABATINGA,AM,CRUZEIRO DO SUL,AC
2,BONITO,PA,CASTANHAL,PA
3,PARAUAPEBAS,PA,XINGUARA,PA
4,CASCAVEL,CE,FORTALEZA,CE
5,JUAZEIRO DO NORTE,CE,BARBALHA,CE
6,PARNAMIRIM,RN,NATAL,RN
7,BAYEUX,PB,CONDE,PB
8,BONITO,PE,PALMARES,PE
9,FERNANDO DE NORONHA,PE,NATAL,RN
